In [1]:
from dataclasses import dataclass
import requests
import json
from typing import Literal
from urllib.parse import urlencode



TwelveDataAPIKEY = "e6944bd6321a49a6a188e0f217667ce0"#config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)


In [7]:
@dataclass
class TwelveDataAPI:
    pair_name: str = "EUR/USD"
    interval: Literal["15min", "1h", "4h", "1day", "1week"] = "15min"
    api_key: str = ""

    def get_api_key(self):
        return self.api_key or TwelveDataAPIKEY

    def get_params(self):
        return {
            "apikey": self.get_api_key(),
            "symbol": self.pair_name,
            "interval": self.interval,
            
        }

    def generate_url(self, pass_auth=False):
        path = "/query"
        url = f"https://api.twelvedata.com/time_series"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&api_key={api_key}"
        return url

    def fetch_data(self):
        headers = {}
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not 200/201
        return response.json()['values']

        

        

In [21]:
stock_api_client = TwelveDataAPI(pair_name='EUR/USD',interval='1day')
dataset = stock_api_client.fetch_data()[0]
dataset

{'datetime': '2025-04-05',
 'open': '1.094811',
 'high': '1.097213',
 'low': '1.093613',
 'close': '1.096251'}

In [20]:
len(dataset)

30

In [ ]:
# parameters that we need to put there

'''
    pair_name = models.CharField(max_length=100, null=True, blank=True)
    open_price = models.DecimalField(max_digits=10, decimal_places=4)
    close_price = models.DecimalField(max_digits=10, decimal_places=4)
    high_price = models.DecimalField(max_digits=10, decimal_places=4)
    low_price = models.DecimalField(max_digits=10, decimal_places=4)
    datetime = models.DateTimeField(null=True, blank=True,max_length=100)
    time_db_update = models.DateTimeField(auto_now=True, null=True, blank=True)
'''

In [15]:
import setup
setup.init_django()
from django.conf import settings
from datadigest.models import pair_1d
from datetime import datetime
from decimal import Decimal


In [19]:
for record in dataset:
    # Convert the date string to a datetime object
    dt = datetime.strptime(record['datetime'], '%Y-%m-%d')
    
    # Create the model instance
    obj = pair_1d.objects.create(
        datetime=dt,
        pair_name = 'EUR/USD',
        open_price=Decimal(record['open']),
        high_price=Decimal(record['high']),
        low_price=Decimal(record['low']),
        close_price=Decimal(record['close'])
    )